In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

from utils import read_data, read_taiwan_specific

import libpysal
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from libpysal.weights import Queen
from scipy.stats import entropy

from utils import column_priorities

待辦
1. 不同縣市的速差、設施影響（使用距離是否大於隨機）
2. 若真的有影響，代表帶入模型應該會有效。但同時GI考慮的是熱點群聚而非事故本身，所以可能不一定有效

模型特徵應該重新篩選，去掉三個設施欄位(停車場、捷運站、youbike)因為他們之前是計算事故附近100公尺內有沒有這個設施，新的方向重新以grid為底去找裡面有多少設施。
同時會計算空間滯後，找鄰近的設施數量來減緩youbike雖然在該grid但是在邊緣區域所造成的偏差

所有特徵：
1. 縣市
2. 設施量
3. 設施空間滯後: 找鄰近的設施數量來減緩設施雖然在該grid但是在邊緣區域所造成的偏差
4. 路網長度
5. 主要風險類型
6. 環境複雜度entropy: 兩個特徵各佔50%，熵會很低、但是如果一個50%，但是有其他很多種類，他的熵會很高。需要處理dominant 的先後順序，如果我只有兩筆事故，應該選無邊線的才對
7. 保留其餘所有特徵但是是以0、1的型態保留而非比例，不然較難以討論
8. 速差

交互作用
1. 特徵、特徵
2. 設施、複雜度（熵）

## 1: County Tagging

In [3]:
combined_data = read_data()
taiwan, grid_filter = read_taiwan_specific(read_grid=True)

grid_gi = pd.read_csv('../ComputedDataV2/Grid/grid_giV3.csv')
grid_gi = grid_gi[grid_gi['num_accidents'] > 0]
grid_gi['COUNTYNAME'] = grid_filter['COUNTYNAME'].astype(str)

grid_gdf = grid_gi.copy() 
county_dummies = pd.get_dummies(grid_gdf['COUNTYNAME'], prefix='County')
grid_gdf = pd.concat([grid_gdf, county_dummies], axis=1)

grid_gdf['geometry'] = grid_gdf['geometry'].apply(wkt.loads)
grid_gdf = gpd.GeoDataFrame(grid_gdf, geometry='geometry')

## 2, 3: Facility Count & Spatial Lag

In [4]:
mrt = pd.read_csv('../ComputedData/MRT/full_mrt.csv')
parking_lot = pd.read_csv('../ComputedData/Parkinglot/full_parkinglot.csv')
youbike = pd.read_csv('../ComputedData/Youbike/full_youbike.csv')

mrt_gdf = gpd.GeoDataFrame(mrt, geometry=gpd.points_from_xy(mrt['PositionLon'], mrt['PositionLat']), crs='EPSG:4326')
parking_lot_gdf = gpd.GeoDataFrame(parking_lot, geometry=gpd.points_from_xy(parking_lot['PositionLon'], parking_lot['PositionLat']), crs='EPSG:4326')
youbike_gdf = gpd.GeoDataFrame(youbike, geometry=gpd.points_from_xy(youbike['PositionLon'], youbike['PositionLat']), crs='EPSG:4326')

def calculate_counts(grid, facility_df, name):
    gdf_fac = gpd.GeoDataFrame(
        facility_df, 
        geometry=gpd.points_from_xy(facility_df['PositionLon'], facility_df['PositionLat']), crs=4326).to_crs(epsg=3826)

    join = gpd.sjoin(gdf_fac, grid[['geometry']], how='inner', predicate='intersects')

    return join.groupby('index_right').size().rename(f'num_{name}')

print("Facilities")
grid_gdf = grid_gdf.join(calculate_counts(grid_gdf, mrt_gdf, 'mrt'), how='left').fillna({'num_mrt': 0})
grid_gdf = grid_gdf.join(calculate_counts(grid_gdf, parking_lot_gdf, 'parking'), how='left').fillna({'num_parking': 0})
grid_gdf = grid_gdf.join(calculate_counts(grid_gdf, youbike_gdf, 'youbike'), how='left').fillna({'num_youbike': 0})

print("Spatial Lag")
w = Queen.from_dataframe(grid_gdf, use_index=True)
grid_gdf['lag_num_mrt'] = libpysal.weights.lag_spatial(w, grid_gdf['num_mrt'])
grid_gdf['lag_num_parking'] = libpysal.weights.lag_spatial(w, grid_gdf['num_parking'])
grid_gdf['lag_num_youbike'] = libpysal.weights.lag_spatial(w, grid_gdf['num_youbike'])

Facilities


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_68667/322713721.py:14: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3826
Right CRS: None

  join = gpd.sjoin(gdf_fac, grid[['geometry']], how='inner', predicate='intersects')
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_68667/322713721.py:14: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3826
Right CRS: None

  join = gpd.sjoin(gdf_fac, grid[['geometry']], how='inner', predicate='intersects')
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_68667/322713721.py:14: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometr

Spatial Lag


/Users/wangqiqian/opt/anaconda3/envs/ST-RTA/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 20964 disconnected components.
 There are 12410 islands with ids: 439, 468, 850, 1459, 1639, 1747, 1827, 2128, 2293, 2395, 2424, 2443, 2670, 3143, 3167, 3274, 3304, 3467, 3574, 3598, 3604, 3635, 3751, 3774, 3907, 3987, 4001, 4008, 4049, 4173, 4226, 4253, 4314, 4321, 4382, 4404, 4416, 4419, 4500, 4602, 4674, 4696, 4710, 4749, 4767, 4934, 5036, 5113, 5205, 5290, 5410, 5471, 5559, 5824, 5827, 5895, 6107, 6179, 6281, 6570, 6583, 6619, 6730, 6741, 6789, 6800, 6886, 6956, 6995, 6998, 7141, 7201, 7206, 7382, 7406, 7418, 7546, 7550, 7584, 7602, 7650, 7827, 8034, 8121, 8175, 8272, 8274, 8325, 8346, 8359, 8391, 8393, 8407, 8416, 8427, 8609, 8619, 8668, 8679, 8811, 8840, 8901, 8932, 8947, 8952, 9095, 9103, 9158, 9175, 9208, 9222, 9262, 9310, 9379, 9399, 9424, 9466, 9472, 9475, 9598, 9668, 9676, 9678, 9680, 9684, 9754, 9777, 97

## 4: Road Network

In [5]:
roads = gpd.read_file("../../ST-RTA-GIS/Data/road_new.shp/gis_osm_roads_free_1.shp")

if isinstance(roads['geometry'].iloc[0], str):
    roads['geometry'] = roads['geometry'].apply(wkt.loads)

roads_gdf = gpd.GeoDataFrame(roads, geometry='geometry', crs=4326).to_crs(epsg=3826)

drive_types = [
    'motorway', 'motorway_link', 
    'trunk', 'trunk_link', 
    'primary', 'primary_link', 
    'secondary', 'secondary_link', 
    'tertiary', 'tertiary_link', 
    'residential', 'unclassified'
]

roads_gdf = roads_gdf[roads_gdf['fclass'].isin(drive_types)]

roads_in_grid = gpd.overlay(roads_gdf, grid_gdf[['geometry']], how='intersection', keep_geom_type=False)
roads_in_grid['segment_length'] = roads_in_grid.length
grid_gdf['grid_id_temp'] = grid_gdf.index 

roads_in_grid = gpd.overlay(roads_gdf, grid_gdf[['geometry', 'grid_id_temp']], how='intersection', keep_geom_type=False)
roads_in_grid['segment_length'] = roads_in_grid.length

grid_road_length = roads_in_grid.groupby('grid_id_temp')['segment_length'].sum().rename('osm_road_length')
grid_gdf = grid_gdf.join(grid_road_length, on='grid_id_temp', how='left')
grid_gdf['osm_road_length'] = grid_gdf['osm_road_length'].fillna(0)
grid_gdf = grid_gdf.drop(columns=['grid_id_temp'])

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_68667/1341942839.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3826
Right CRS: None

  roads_in_grid = gpd.overlay(roads_gdf, grid_gdf[['geometry']], how='intersection', keep_geom_type=False)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_68667/1341942839.py:23: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3826
Right CRS: None

  roads_in_grid = gpd.overlay(roads_gdf, grid_gdf[['geometry', 'grid_id_temp']], how='intersection', keep_geom_type=False)


## 5,6,7

5. 主要風險類型
6. 環境複雜度entropy: 兩個特徵各佔50%，熵會很低、但是如果一個50%，但是有其他很多種類，他的熵會很高。需要處理dominant 的先後順序，如果我只有兩筆事故，應該選無邊線的才對
7. 保留其餘所有特徵但是是以0、1的型態保留而非比例，不然較難以討論

In [8]:
combined_data = gpd.GeoDataFrame(combined_data, geometry=gpd.points_from_xy(combined_data['經度'], combined_data['緯度']), crs='EPSG:4326')

def calculate_single_column_metrics(series, priority_map):

    if len(series) == 0:
        return pd.Series([None, 0.0, 0.0], index=['dom', 'ratio', 'ent'])

    counts = series.value_counts()
    total = len(series)
    
    # Entropy
    probs = counts / total
    ent = entropy(probs, base=np.e)
    
    # Dominant
    df_temp = pd.DataFrame({
        'type': counts.index,
        'count': counts.values,
        'priority': [priority_map.get(t, 0) for t in counts.index] # 查不到給0分
    })

    df_temp = df_temp.sort_values(by=['count', 'priority'], ascending=[False, False])
    
    dom_type = df_temp.iloc[0]['type']
    dom_count = df_temp.iloc[0]['count']
    dom_ratio = dom_count / total
    
    return pd.Series([dom_type, dom_ratio, ent], index=['dom', 'ratio', 'ent'])

In [9]:
grid_gdf.set_crs(epsg=3826, inplace=True, allow_override=True)
combined_data = combined_data.to_crs(epsg=3826)

In [10]:
joined = gpd.sjoin(combined_data, grid_gdf[['geometry']], how='inner', predicate='intersects')
joined['grid_id'] = joined.index_right

results = []

for col, p_map in column_priorities.items():
    print(col)

    metrics = joined.groupby('grid_id')[col].apply(
        lambda x: calculate_single_column_metrics(x, p_map)
    ).unstack()

    simple_name = col.split('-')[-1].replace('名稱', '') 
    metrics.columns = [f'{simple_name}_{c}' for c in metrics.columns]
    
    results.append(metrics)

車道劃分設施-分道設施-快車道或一般車道間名稱
車道劃分設施-分道設施-快慢車道間名稱
車道劃分設施-分道設施-路面邊線名稱
車道劃分設施-分向設施大類別名稱
事故類型及型態子類別名稱
道路型態子類別名稱
號誌-號誌種類名稱


In [ ]:
# 7. 保留其餘所有特徵但是是以0、1的型態保留而非比例，不然較難以討論
binary_results = []

for col in column_priorities.keys():
    crosstab = pd.crosstab(joined['grid_id'], joined[col])

    flag_df = (crosstab > 0).astype(int)

    simple_name = col.split('-')[-1].replace('名稱', '')
    flag_df.columns = [f'{simple_name}_{c}' for c in flag_df.columns]
    
    binary_results.append(flag_df)

all_binary_flags = pd.concat(binary_results, axis=1)

categorical_features = pd.concat(results, axis=1)
all_features = pd.concat([categorical_features, all_binary_flags], axis=1)

final_grid_features = grid_gdf.join(all_features, how='left')

speed_col = '速限-第1當事者'
valid_speed = joined[joined[speed_col] > 0].copy()

speed_metrics = valid_speed.groupby('grid_id')[speed_col].agg(['max', 'mean', 'std'])
speed_metrics.columns = ['Speed_Max', 'Speed_Mean', 'Speed_Std']
speed_metrics['Speed_Std'] = speed_metrics['Speed_Std'].fillna(0)

final_grid_features = final_grid_features.join(speed_metrics, how='left')
fill_cols = [c for c in final_grid_features.columns if any(x in c for x in ['ratio', 'ent', 'Std', 'Mean', 'Has_'])]
final_grid_features[fill_cols] = final_grid_features[fill_cols].fillna(0)

print(final_grid_features.columns.tolist())

['geometry', 'num_accidents', 'accident_indices', 'centroid', 'GiZScore', 'GiPValue', 'hotspot', 'COUNTYNAME', 'County_nan', 'County_南投縣', 'County_嘉義市', 'County_嘉義縣', 'County_基隆市', 'County_宜蘭縣', 'County_屏東縣', 'County_彰化縣', 'County_新北市', 'County_新竹市', 'County_新竹縣', 'County_桃園市', 'County_臺中市', 'County_臺北市', 'County_臺南市', 'County_臺東縣', 'County_花蓮縣', 'County_苗栗縣', 'County_雲林縣', 'County_高雄市', 'num_mrt', 'num_parking', 'num_youbike', 'lag_num_mrt', 'lag_num_parking', 'lag_num_youbike', 'osm_road_length', '快車道或一般車道間_dom', '快車道或一般車道間_ratio', '快車道或一般車道間_ent', '快慢車道間_dom', '快慢車道間_ratio', '快慢車道間_ent', '路面邊線_dom', '路面邊線_ratio', '路面邊線_ent', '分向設施大類別_dom', '分向設施大類別_ratio', '分向設施大類別_ent', '事故類型及型態子類別_dom', '事故類型及型態子類別_ratio', '事故類型及型態子類別_ent', '道路型態子類別_dom', '道路型態子類別_ratio', '道路型態子類別_ent', '號誌種類_dom', '號誌種類_ratio', '號誌種類_ent', '快車道或一般車道間_未繪設車道線', '快車道或一般車道間_禁止變換車道線(無標記)', '快車道或一般車道間_禁止變換車道線(附標記)', '快車道或一般車道間_車道線(無標記)', '快車道或一般車道間_車道線(附標記)', '快慢車道間_寬式快慢車道分隔島(50公分以上)', '快慢車道間_快慢車道分隔線', '快慢車道間_未繪設快慢車道分隔

## 8: Speed Difference

In [12]:
pairs_annot_all_cities = gpd.read_file("../../ST-RTA-GIS/CalculatedData/pairs_annot_all_cities.shp").to_crs(epsg=3826)

spd_join = gpd.sjoin(
    pairs_annot_all_cities[['spd_dlt', 'geometry']], 
    grid_gdf[['geometry']], 
    how='inner', 
    predicate='intersects'
)
spd_mean = spd_join.groupby(spd_join.index_right)['spd_dlt'].mean().rename('spd_dlt_mean')

final_grid_features['spd_dlt_mean'] = spd_mean
final_grid_features['spd_dlt_mean'] = final_grid_features['spd_dlt_mean'].fillna(0)

In [ ]:
['geometry', 'num_accidents', 'accident_indices', 'centroid', 'GiZScore', 'GiPValue', 'hotspot', 'COUNTYNAME',
 # 1
 'County_nan', 'County_南投縣', 'County_嘉義市', 'County_嘉義縣', 'County_基隆市', 'County_宜蘭縣', 'County_屏東縣', 'County_彰化縣', 'County_新北市', 'County_新竹市', 'County_新竹縣', 'County_桃園市', 'County_臺中市', 'County_臺北市', 'County_臺南市', 'County_臺東縣', 'County_花蓮縣', 'County_苗栗縣', 'County_雲林縣', 'County_高雄市', 
 # 2
 'num_mrt', 'num_parking', 'num_youbike', 
 # 3
 'lag_num_mrt', 'lag_num_parking', 'lag_num_youbike',
 # 4
 'osm_road_length', 
 # 5, 6, 7
 '快車道或一般車道間_dom', '快車道或一般車道間_ratio', '快車道或一般車道間_ent', 
 '快慢車道間_dom', '快慢車道間_ratio', '快慢車道間_ent', 
 '路面邊線_dom', '路面邊線_ratio', '路面邊線_ent', 
 '分向設施大類別_dom', '分向設施大類別_ratio', '分向設施大類別_ent', 
 '事故類型及型態子類別_dom', '事故類型及型態子類別_ratio', '事故類型及型態子類別_ent', 
 '道路型態子類別_dom', '道路型態子類別_ratio', '道路型態子類別_ent', 
 '號誌種類_dom', '號誌種類_ratio', '號誌種類_ent', 
 # 8
 'Speed_Mean', # ＋其餘道路特徵
#  'Speed_Max', 'Speed_Std',
 # 9
 'spd_dlt_mean']

In [13]:
final_grid_features.to_csv('../ComputedDataV6/ForModel/final_grid_features.csv', index=False)

In [159]:
dom_cols = [c for c in final_grid_features.columns if c.endswith('_dom')]
final_grid_features = pd.get_dummies(final_grid_features, columns=dom_cols)

In [168]:
final_features = final_grid_features[final_grid_features['County_nan'] != 1]
final_features = final_features.drop(columns=['COUNTYNAME', 'geometry', 'num_accidents', 'accident_indices', 'centroid', 'GiZScore', 'GiPValue', 'hotspot', 'Speed_Max', 'Speed_Std', 'County_nan'])